In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine

# Configura tus credenciales de conexión
usuario = 'root'
contraseña = 'isabelyluis'
host = 'localhost'
base_de_datos = 'youth_housing'

# Crea el motor SQLAlchemy
engine = create_engine(f'mysql+pymysql://{usuario}:{"isabelyluis"}@{host}/{'youth_housing'}')


In [4]:
# Ruta de salida
output_folder = "../data/final_csvs"
os.makedirs(output_folder, exist_ok=True)

# Diccionario con nombre de archivo y su query corregida
queries = {
    "h1_esfuerzo_ccaa.csv": """
        SELECT 
            p.comunidades_y_ciudades_autónomas,
            p.año,
            p.precio_m2,
            s.total,
            ROUND(s.total, 2) AS salario_mensual,
            ROUND(p.precio_m2 / (s.total), 2) AS esfuerzo_m2_mensual
        FROM precios p
        JOIN salario_ccaa s
          ON p.comunidades_y_ciudades_autónomas = s.comunidades_y_ciudades_autónomas AND p.año = s.año
        WHERE s.decil = 'Total decil'
          AND s.tipo_de_jornada = 'Total'
    """,
    "h1_ranking_ccaa.csv": """
        SELECT 
            comunidades_y_ciudades_autónomas,
            ROUND(AVG(esfuerzo_m2_mensual), 2) AS esfuerzo_medio
        FROM (
            SELECT 
                p.comunidades_y_ciudades_autónomas,
                p.año,
                p.precio_m2,
                s.total,
                ROUND(s.total, 2) AS salario_mensual,
                ROUND(p.precio_m2 / (s.total), 2) AS esfuerzo_m2_mensual
            FROM precios p
            JOIN salario_ccaa s
              ON p.comunidades_y_ciudades_autónomas = s.comunidades_y_ciudades_autónomas 
             AND p.año = s.año
            WHERE s.decil = 'Total decil'
              AND s.tipo_de_jornada = 'Total'
        ) AS subconsulta
        GROUP BY comunidades_y_ciudades_autónomas
        ORDER BY esfuerzo_medio ASC
    """,
    "h1_ranking_actual.csv": """
        SELECT 
            p.comunidades_y_ciudades_autónomas,
            ROUND(p.precio_m2 / (s.total),2) AS esfuerzo_m2_mensual
        FROM precios p
        JOIN salario_ccaa s
          ON p.comunidades_y_ciudades_autónomas = s.comunidades_y_ciudades_autónomas 
         AND p.año = s.año
        WHERE s.decil = 'Total decil'
          AND s.tipo_de_jornada = 'Total'
          AND p.año = 2023
        ORDER BY esfuerzo_m2_mensual ASC
    """,
    "h3_esfuerzo_joven_vs_general.csv": """
        SELECT 
            p.comunidades_y_ciudades_autónomas,
            p.año,
            p.precio_m2,
            sj.total AS salario_mensual_joven,
            ROUND(p.precio_m2 / sj.total, 2) AS esfuerzo_m2_joven,
            sg.total AS salario_mensual_general,
            ROUND(p.precio_m2 / sg.total, 2) AS esfuerzo_m2_general
        FROM precios p
        JOIN salario_edad sj ON p.año = sj.año
        JOIN salario_ccaa sg ON p.año = sg.año AND p.comunidades_y_ciudades_autónomas = sg.comunidades_y_ciudades_autónomas
        WHERE sj.grupo_de_edad = 'De 25 a 34 años'
          AND sj.tipo_de_jornada = 'Total'
          AND sj.decil = 'Total decil'
          AND sg.tipo_de_jornada = 'Total'
          AND sg.decil = 'Total decil'
    """,
    "h3_metros_vs_superficie.csv": """
        SELECT 
            p.comunidades_y_ciudades_autónomas,
            p.año,
            p.precio_m2,
            sj.total AS salario_mensual_joven,
            s.superficie_util,
            ROUND(sj.total / p.precio_m2, 2) AS metros_posibles,
            ROUND((sj.total / p.precio_m2) - s.superficie_util, 2) AS diferencia_m2
        FROM precios p
        JOIN superficie_ok s ON p.año = s.año AND p.comunidades_y_ciudades_autónomas = s.comunidades_y_ciudades_autónomas
        JOIN salario_edad sj ON p.año = sj.año
        WHERE sj.grupo_de_edad = 'De 25 a 34 años'
          AND sj.tipo_de_jornada = 'Total'
          AND sj.decil = 'Total decil'
    """,
    "h4_interes_precio_esfuerzo.csv": """
        SELECT 
            i.año,
            ROUND(AVG(i.total), 2) AS interes_medio,
            ROUND(AVG(p.precio_m2), 2) AS precio_m2_medio,
            ROUND(AVG(s.total), 2) AS salario_mensual_medio,
            ROUND(AVG(p.precio_m2) / AVG(s.total), 2) AS esfuerzo_m2_mensual
        FROM interes_fijo i
        JOIN precios p ON i.año = p.año
        JOIN salario_ccaa s ON i.año = s.año
        WHERE i.tipo_de_interés = 'Fijo'
          AND i.naturaleza_de_la_finca = 'Viviendas'
          AND s.tipo_de_jornada = 'Total'
          AND s.decil = 'Total decil'
        GROUP BY i.año
        ORDER BY i.año
    """,
    "h5_variacion_precios.csv": """
        SELECT 
            comunidad.comunidades_y_ciudades_autónomas,
            MIN(CASE WHEN comunidad.año = 2020 THEN comunidad.precio_m2 END) AS precio_2020,
            MAX(CASE WHEN comunidad.año = 2024 THEN comunidad.precio_m2 END) AS precio_2024,
            ROUND(
                MAX(CASE WHEN comunidad.año = 2024 THEN comunidad.precio_m2 END) -
                MIN(CASE WHEN comunidad.año = 2020 THEN comunidad.precio_m2 END), 2
            ) AS variacion_absoluta,
            ROUND(
                100 * (
                    MAX(CASE WHEN comunidad.año = 2024 THEN comunidad.precio_m2 END) -
                    MIN(CASE WHEN comunidad.año = 2020 THEN comunidad.precio_m2 END)
                ) / MIN(CASE WHEN comunidad.año = 2020 THEN comunidad.precio_m2 END), 2
            ) AS variacion_porcentual
        FROM precios comunidad
        WHERE año IN (2020, 2024)
        GROUP BY comunidad.comunidades_y_ciudades_autónomas
        ORDER BY variacion_porcentual DESC
    """
}

# Ejecutar queries y exportar a CSVs
for filename, query in queries.items():
    df = pd.read_sql_query(query, engine)
    df.to_csv(os.path.join(output_folder, filename), index=False, decimal=".")
    print(f"{filename} exportado correctamente.")

h1_esfuerzo_ccaa.csv exportado correctamente.
h1_ranking_ccaa.csv exportado correctamente.
h1_ranking_actual.csv exportado correctamente.
h3_esfuerzo_joven_vs_general.csv exportado correctamente.
h3_metros_vs_superficie.csv exportado correctamente.
h4_interes_precio_esfuerzo.csv exportado correctamente.
h5_variacion_precios.csv exportado correctamente.
